In [31]:
from lxml import etree
from lxml import objectify
import pandas as pd
import re as re
from sqlalchemy import create_engine
from subprocess import Popen
#import lxml.usedoctest from http://lxml.de/objectify.html

In [32]:
p_out_sql_cat='Cat'
p_out_sql_id='VarID'
p_out_sql_answer='AnswerLabel'
p_out_sql_lists='Lists'

In [33]:
path_mdd=r'\\TSHAMFIL901\Work\DV\MatthiasH\BBG_Tracking'.replace(chr(92),'/')
file_mdd=path_mdd+'/'+'R315114383A.mdd'

pfad_out=r'\\TSHAMFIL901\Work\DV\MatthiasH\BBG_Tracking\out'.replace(chr(92),'/')
file_cat_out_csv=pfad_out+'/'+p_out_sql_cat+'.csv'
file_id_out_csv=pfad_out+'/'+p_out_sql_id+'.csv'
file_answer_out_csv=pfad_out+'/'+p_out_sql_answer+'.csv'
file_lists_out_csv=pfad_out+'/'+p_out_sql_lists+'.csv'


path_bat=r'\\TSHAMFIL901\Work\DV\MatthiasH\BBG_Tracking\out'+chr(92)
file_bat=path_bat+'/'+'bbg_csv.bat'

In [34]:
p_mode='r' # r=replace, a = append

In [35]:
engine_dv_bbg = create_engine('mssql+pyodbc://TSMMHSQVS901/DV_BBG?driver=SQL+Server+Native+Client+11.0?trusted_connection=yes')
p_if_exists='fail'
if p_mode=='r':
    p_if_exists='replace'
if p_mode=='a':
    p_if_exists='append'

In [36]:
f = open(file_mdd,'r', encoding="utf8")
filedata = f.read()
f.close()
filecontent=filedata.replace('/Arc 3/2000-02-04','/Arc_3/2000_02_04')
filecontent=filecontent.replace('encoding="UTF-8"','')
tree = etree.fromstring(filecontent)

In [37]:
list_variables=tree.findall('.//definition/variable')
list_categories=tree.findall('.//definition/categories')
list_fields=tree.findall('.//design/fields')
type(list_categories[0])
list_fields[0].attrib

{'global-name-space': '-1', 'name': '@fields'}

In [38]:
dfid = pd.DataFrame()
for field in list_fields:
    #for variable in field.getchildren():
    for variable in field.iter("variable"):
        #print(variable.attrib['id'],variable.attrib['name'])
        dfid = dfid.append(pd.Series([variable.attrib['id'].replace('_',''),variable.attrib['name'],'variable']),ignore_index=True)
    for loop in field.iter("loop"):
        for class_ in loop.iter("class"):
            for fields in class_.iter("fields"):
                for variable in fields.iter("variable"):
        #print(loop.attrib['id'],loop.attrib['name'])
                    dfid = dfid.append(pd.Series([variable.attrib['id'].replace('_',''),loop.attrib['name'],'loop']),ignore_index=True)
dfid.columns=['id','name','type']

In [39]:
dfid.to_csv(file_id_out_csv,sep='\t',index=False)
try:
    dfid[:0].to_sql(p_out_sql_id,engine_dv_bbg, if_exists=p_if_exists, index=False)
except ValueError:
    pass

#dfBTS[dfdb.columns].to_csv(p_out_csv_L1_BTS_country,sep='\t',index=False)
p = Popen([file_bat,p_out_sql_id,p_out_sql_id+'.csv'], cwd=path_bat)
stdout, stderr = p.communicate()

In [40]:
df = pd.DataFrame()
for variable in list_variables:
    for categories in variable.getchildren():
        for category in categories.iter("category"):
            for labels in category.iter("labels"):
                for element in labels.iter("text"):
                    if element.attrib['{http://www.w3.org/XML/1998/namespace}lang']=='de-DE':
                        #print(variable.attrib['name'],category.attrib['name'], element.text)
                        df = df.append(pd.Series([variable.attrib['id'].replace('_',''),category.attrib['id'],variable.attrib['name'],category.attrib['name'], element.text]),ignore_index=True)
df.columns=['DimID','ID','DimVar',  'DimVal','Label']

In [41]:
dfx = pd.DataFrame()
for categories_ in list_categories:
#    for categories_ in categories_.getchildren():
    for category in categories_.iter("category"):
        for labels in category.iter("labels"):
            for element in labels.iter("text"):
                if element.attrib['{http://www.w3.org/XML/1998/namespace}lang']=='de-DE':
                    #print(variable.attrib['name'],category.attrib['name'], element.text)
                    dfx = dfx.append(pd.Series([variable.attrib['id'].replace('_',''),category.attrib['id'],categories_.attrib['name'],category.attrib['name'], element.text]),ignore_index=True)
dfx.columns=['DimID','ID','DimVar',  'DimVal','Label']

In [42]:
#df[['DimVar','DimVal','Label']].to_csv(file_answer_out_csv,sep='\t',index=False)
dfx.to_csv(file_lists_out_csv,sep='\t',index=False)
try:
    dfx[:0].to_sql(p_out_sql_lists,engine_dv_bbg, if_exists=p_if_exists, index=False)
except ValueError:
    pass

#dfBTS[dfdb.columns].to_csv(p_out_csv_L1_BTS_country,sep='\t',index=False)
p = Popen([file_bat,p_out_sql_lists,p_out_sql_lists+'.csv'], cwd=path_bat)
stdout, stderr = p.communicate()

In [43]:
print(df[['DimVar','DimVal','Label']].to_string())

               DimVar             DimVal                                              Label
0        zzFlagSample                 _1                                                LSR
1        zzFlagSample                 _2                                                GMI
2        zzFlagSample                 _3                                             Toluna
3              zzwave              wave1                                            Welle 1
4              zzwave              wave2                                            Welle 2
5              zzwave              wave3                                            Welle 3
6             zztotal                 _1                                              Total
7                jump                 _1                                                 ja
8                jump                 _2                                               nein
9                  S0                 _1                                        

In [58]:
df.DimVar.unique()

array(['zzFlagSample', 'zzwave', 'zztotal', 'jump', 'S0', 'zzgender',
       'zzAge', 'S2', 'zzregion', 'hwestost', 'S4', 'S5', 'S6', 'S7',
       'slice', 'S9a', 'zzMainType', 'zzMainUser', 'zzSchwarzbier',
       'zzTestGruppe', 'Q3a', 'Q4a', 'Q4aa', 'Q4ab', 'Q5a', 'Q8a', 'Q9a',
       'Q9b', 'Q13b', 'Q14b_1', 'Q17b', 'MDN3', 'SD1', 'SD2', 'SD3', 'SD4',
       'comp', 'TrapStatus', 'kantarDevice', 'Match', 'Gender', 'sampsrc',
       'submitButtonText', 'endText', 'globalOptions', 'engVersion',
       'manifest', 'location', 'qweek', 'q1', 'q2', 'Origin', 'Status',
       'RoutingContext', 'DataEntryMode', 'ReviewStatus'], dtype=object)

In [44]:
#df[['DimVar','DimVal','Label']].to_csv(file_answer_out_csv,sep='\t',index=False)
df.to_csv(file_answer_out_csv,sep='\t',index=False)
try:
    df[:0].to_sql(p_out_sql_answer,engine_dv_bbg, if_exists=p_if_exists, index=False)
except ValueError:
    pass

#dfBTS[dfdb.columns].to_csv(p_out_csv_L1_BTS_country,sep='\t',index=False)
p = Popen([file_bat,p_out_sql_answer,p_out_sql_answer+'.csv'], cwd=path_bat)
stdout, stderr = p.communicate()

In [45]:
list_categoryids=tree.findall('.//categorymap/categoryid')

dfcat = pd.DataFrame()
for categoryid in list_categoryids:
    dfcat = dfcat.append(pd.Series([categoryid.attrib['value'],categoryid.attrib['name']]),ignore_index=True)
dfcat.columns=['CatValue','CatName']
dfcat.CatValue=dfcat.CatValue.astype(int)

varcode=re.compile('^(_[0-9]{1,7})$')
dfcat['Code']=dfcat.CatName.str.extract(varcode, expand=True)[0].str[1:].fillna(-1).astype(int)
varcode=re.compile('(.*?)(_[0-9]{1,7})(.*?)')
dfcat['Code2']=dfcat.CatName.str.extract(varcode, expand=True)[1].str[1:].fillna(-1).astype(int)

dict_mdd=dict(zip(dfcat['CatValue'].astype(int),dfcat['CatName']))

In [46]:
dfcat.to_csv(file_cat_out_csv,sep='\t',index=False)
try:
    dfcat[:0].to_sql(p_out_sql_cat,engine_dv_bbg, if_exists=p_if_exists, index=False)
except ValueError:
    pass

#dfBTS[dfdb.columns].to_csv(p_out_csv_L1_BTS_country,sep='\t',index=False)
p = Popen([file_bat,p_out_sql_cat,p_out_sql_cat+'.csv'], cwd=path_bat)
stdout, stderr = p.communicate()

In [13]:
dict_mdd

{1: 'scan',
 2: 'htmlplayer',
 3: 'cati_htmlplayer',
 4: 'cati_ttyplayer',
 5: 'other',
 6: 'completed',
 7: 'active',
 8: 'timedout',
 9: 'scriptstopped',
 10: 'respondentstopped',
 11: 'shutdown',
 12: 'reviewed',
 13: 'signal',
 14: 'test',
 15: 'paper',
 16: 'sif',
 17: 'clean',
 18: 'needsreview',
 19: 'flaggedforreview',
 20: 'flaggedformove',
 21: 'flaggedforcopy',
 22: 'flaggedfordelete',
 23: 'dataentryplayer',
 24: 'initial',
 25: 'full',
 26: 'partial',
 27: 'maintenance',
 28: 'web',
 29: 'compclosed5',
 30: 'compclosed6',
 31: 'compclosed7',
 32: 'compclosed8',
 33: 'compclosed9',
 34: 'incomplete',
 35: 'incompatible',
 36: 'overquota',
 37: 'trapscreenout',
 38: 'tnsscreenout',
 39: 'ts_notengaged',
 40: 'enabled',
 41: 'disabled',
 42: 'pc',
 43: 'largetablet',
 44: 'largefeaturephone',
 45: 'mediumtablet',
 46: 'smalltablet',
 47: 'smallfeaturephone',
 48: 'smartphonetouch',
 49: 'unknown',
 50: 'yes',
 51: 'no',
 52: 'm',
 53: 'f',
 54: 'ipl01',
 55: 'ssi02',
 56: 'er

In [23]:
p_file_csv=p_pfad+'/clean3.csv'
df.to_csv(p_file_csv,sep='\t',index=False)

In [24]:
p_file_xlsx=p_pfad+'/clean3.xlsx'
df.to_excel(p_file_xlsx,index=False)

In [25]:
dfp = pd.DataFrame()

for categories in list_categories:
    for category in categories.iter("category"):
        for properties in category.iter("properties"):
            for prop in properties.iter("property"):
                try:
                    dfp = dfp.append(pd.Series([category.attrib['id'],prop.attrib['name'],prop.attrib['value']]),ignore_index=True)
                except (NameError, KeyError):
                    pass
dfp.columns=['ID','PropVar','PropVal']

In [26]:
dfpp=dfp.pivot(index='ID', columns='PropVar', values='PropVal').copy()

In [27]:
dfi=df.set_index(['ID'])

In [28]:
dfpp=pd.merge(dfi, dfpp, how='outer', on=None, left_on=None, right_on=None,
         left_index=True, right_index=True, sort=False,
         suffixes=('_x', '_y'), copy=True, indicator=False)

In [31]:
dfc = pd.DataFrame()
for fields in list_fields:
    for fields_class in fields.iter("class"):
        for class_fields in fields_class.iter("fields"):
            for element in class_fields.iter("variable"):
                dfc = dfc.append(pd.Series([fields_class.attrib['name'], element.attrib['id'].replace('_','')]),ignore_index=True)
dfc.columns=['ClassName','DimID']

In [32]:
dfc

,ClassName,DimID
0,Admin,89491e0d-ba31-43ad-babf-a695b996b608
1,Admin,d402afb7-22d7-4df7-848e-4fcdc0783972
2,Admin,ab143b5d-a93f-4e07-a72b-73d5a366e318
3,Admin,05a474ae-02d8-4736-881b-2a025dd87fc2
4,Admin,f25843ba-9bc9-4552-93e2-890dd8595596
5,Admin,a416cda1-a2c8-43d0-acd2-b54e4183b174
6,Admin,933b06e6-b214-4dd7-8206-359f4ea6d51c
7,Income,b28261db-c42c-4bb5-9d6f-1ce163868cd6
8,Edu,37e56f75-8966-41c8-9fab-7a5e13f6be08
9,Region,a6bbf4ea-acae-481c-9157-0c85481e3730


In [33]:
dfppc=pd.merge(dfpp, dfc, how='outer', on='DimID',
         sort=False,
         suffixes=('_x', '_y'), copy=True, indicator=False)

In [34]:
p_file_xlsx=p_pfad+'/clean3properties.xlsx'
dfppc.to_excel(p_file_xlsx,index=False)